# Clustering utility functions

In [1]:
import numpy as np
from numpy.random import uniform
from random import sample
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from kneed import KneeLocator
from sklearn.metrics import silhouette_score




# Python Implementation for computing Hopkins' Statistic:

# function to compute hopkins's statistic for the dataframe X
def hopkins_statistic(X):
    
    X=X.values  #convert dataframe to a numpy array
    sample_size = int(X.shape[0]*0.05) #0.05 (5%) based on paper by Lawson and Jures
    
    
    #a uniform random sample in the original data space
    X_uniform_random_sample = uniform(X.min(axis=0), X.max(axis=0) ,(sample_size , X.shape[1]))
    
    #a random sample of size sample_size from the original data X
    random_indices=sample(range(0, X.shape[0], 1), sample_size)
    X_sample = X[random_indices]
   
    
    #initialise unsupervised learner for implementing neighbor searches
    neigh = NearestNeighbors(n_neighbors=2)
    nbrs=neigh.fit(X)
    
    #u_distances = nearest neighbour distances from uniform random sample
    u_distances , u_indices = nbrs.kneighbors(X_uniform_random_sample , n_neighbors=2)
    u_distances = u_distances[: , 0] #distance to the first (nearest) neighbour
    
    #w_distances = nearest neighbour distances from a sample of points from original data X
    w_distances , w_indices = nbrs.kneighbors(X_sample , n_neighbors=2)
    #distance to the second nearest neighbour (as the first neighbour will be the point itself, with distance = 0)
    w_distances = w_distances[: , 1]
    
 
    
    u_sum = np.sum(u_distances)
    w_sum = np.sum(w_distances)
    
    #compute and return hopkins' statistic
    H = u_sum/ (u_sum + w_sum)
    return H

In [2]:
#Utility to sort cluster basing on the number of points contained.
def orderClusterDescending(numpyArray):
    
    unique_elements, counts = np.unique(numpyArray, return_counts=True)
    
    # Create a dictionary to store the counts by element
    count_dict = dict(zip(unique_elements, counts))
    
    # Sort the dictionary by values and get the sorted keys
    sorted_keys = sorted(count_dict, key=count_dict.get,reverse=True)

    # Create a list of indexes representing the sorted order
    indexes = [list(count_dict.keys()).index(key) for key in sorted_keys] 
    
    #for element, count in count_dict.items():

    copy2=numpyArray.copy()

    for i in range(0,len(numpyArray)-1):
        elem=numpyArray[i]
        copy2[i]=indexes.index(elem)
    
    return copy2

# K-Means

In [5]:
# Implementation of elbow method to inspect which number of k is better to select for k-means algoritm

def optimise_k_means(data, max_k):
    means=[]
    inertias=[]
    
    for k in range(1,max_k):
        kmeans=KMeans(n_clusters=k)
        kmeans.fit(data)
        means.append(k)
        inertias.append(kmeans.inertia_)
        
    #Generate the elbow plot 
    fig=plt.subplots(figsize=(10,5))
    plt.plot(means,inertias,'o-')
    plt.xlabel('Number of clusters')
    plt.ylabel('Inertia')
    plt.grid(True)
    plt.show()
    

# DBSCAN

In [ ]:
#Using the method of the knee to discover what is the best value for eps.

def find_opt_eps(data):

    # n_neighbors = 5 as kneighbors function returns distance of point to itself (i.e. first column will be zeros) 
    nbrs = NearestNeighbors(n_neighbors = 4).fit(data.iloc[:, :])
    # Find the k-neighbors of a point
    neigh_dist, neigh_ind = nbrs.kneighbors(data.iloc[:, :])
    # sort the neighbor distances (lengths to points) in ascending order
    # axis = 0 represents sort along first axis i.e. sort along row
    sort_neigh_dist = np.sort(neigh_dist, axis = 0)
    
    k_dist = sort_neigh_dist[:, 3]
    plt.plot(k_dist)
    plt.ylabel("k-NN distance")
    plt.xlabel("Sorted observations (4th NN)")
    plt.show()
    
    kneedle = KneeLocator(x = range(1, len(neigh_dist)+1), y = k_dist, S = 1.0, 
                      curve = "concave", direction = "increasing", online=True)

    # get the estimate of knee point
    #print(kneedle.knee_y)

    kneedle.plot_knee()
    plt.show()
    return kneedle.knee_y

In [ ]:
# Grid search for optimizing the silhutette index given a range of eps and min_points
def gridSearchOptCluster(data,list_eps,list_minPoints):
    
    best_eps=-1
    best_minPoints=-1
    best_labels=[]
    max_siluette=-1
    
    for minPoints in list_minPoints:
       
        for eps in list_eps:
            #print ("minPoints nr : "+str(minPoints)+" eps: "+str(eps))
            clusters = DBSCAN(eps = eps, min_samples = minPoints).fit(data.iloc[:, :])
            score = silhouette_score(data.iloc[:, :], clusters.labels_, metric='euclidean')
            if (score>max_siluette):
                max_siluette=score
                best_labels=clusters.labels_
                best_eps=eps
                best_minPoints=minPoints
                #print ("Best score found: "+str(score)+" eps: "+str(eps)+" minPoints: "+str(minPoints))
    return {"best_score": max_siluette,"best_eps": eps, "best_minPoints":best_minPoints}